In [ ]:
%load_ext kedro.ipython

In [ ]:
from pathlib import Path
import typing as t
from IPython.display import Markdown

from kedro.config import OmegaConfigLoader
from kedro.framework.project import settings
from google import genai
from google.genai import types

In [ ]:
conf_path = str(Path("..") / settings.CONF_SOURCE)
conf_loader = OmegaConfigLoader(conf_source=conf_path)
GOOGLE_API_KEY = conf_loader["credentials"]["google_api_credentials"]["key"]

In [ ]:
cv: dict[str, t.Any] = catalog.load("resume")  # noqa: F821

In [ ]:
client = genai.Client(api_key=GOOGLE_API_KEY)

In [ ]:
LANG = "es"

# The selected topic.
#  Recomended topics: "work", "certificates", "publications", "projects"
TOPIC = "work"

# Number maximum of experiences to generate for the selected topic.
# Recommended values:
#  - work: 3
#  - certificates: 2
#  - publications: 2
#  - projects: 2
N_MAX_EXP = 3

In [ ]:
# Aliasing, only for readability
type TopicName = str
type TopicDict = dict[str, str | list[str]]
type Resume = dict[TopicName, TopicDict]
type Document = str
type Documents = list[Document]
type IDs = list[str]


class DocumentsIDs(t.TypedDict):
    documents: Documents
    ids: IDs


GENERATE_ID_DICT: dict[TopicName, t.Callable[[TopicDict], str]] = {
    "work": lambda item: f"{item['name']}.{item['position']}",
    "certificates": lambda item: f"{item['issuer']}.{item['name']}",
    "publications": lambda item: f"{item['publisher']}.{item['name']}",
    "projects": lambda item: item["name"],
    "volunteer": lambda item: item["position"],
    "education": lambda item: f"{item['studyType']}.{item['area']}",
    "basics": lambda item: item["name"],
    "awards": lambda item: f"{item['title']}.{item['awarder']}",
    "skills": lambda item: item["name"],
    "languages": lambda item: item["language"],
    "interests": lambda item: item["name"],
    "references": lambda item: item["name"],
}


def decorate_gen_id(
    topic: TopicName, id_fn: t.Callable[[TopicDict], str]
) -> t.Callable[[TopicDict], str]:
    """
    Decorator to generate IDs for documents.

    It adds the topic name to the ID generated by the provided function. It also
    replaces spaces, dashes, commas, and colons with underscores. This is useful
    for creating unique IDs for documents in a structured format.
    """

    def wrapper(item: TopicDict) -> str:
        id_fn_result = id_fn(item)

        # Clear any special characters from the ID
        id_fn_result = (
            id_fn_result.replace(" ", "_")
            .replace("-", "_")
            .replace(",", "_")
            .replace(":", "_")
        )
        id_fn_result = (
            id_fn_result.replace("(", "")
            .replace(")", "")
            .replace("'", "")
            .replace('"', "")
        )
        id_fn_result = id_fn_result.replace(
            "..", "."
        )  # Chromadb do not accept double dots

        # Clear special characters
        id_fn_result = (
            id_fn_result.replace("á", "a")
            .replace("é", "e")
            .replace("í", "i")
            .replace("ó", "o")
            .replace("ú", "u")
        )
        id_fn_result = (
            id_fn_result.replace("Á", "A")
            .replace("É", "E")
            .replace("Í", "I")
            .replace("Ó", "O")
            .replace("Ú", "U")
        )
        id_fn_result = (
            id_fn_result.replace("ñ", "n")
            .replace("ü", "u")
            .replace("Ñ", "N")
            .replace("Ü", "U")
        )

        return topic + "." + id_fn_result

    return wrapper


GENERATE_ID_DICT = {
    topic: decorate_gen_id(topic, id_fn) for topic, id_fn in GENERATE_ID_DICT.items()
}

In [ ]:
import yaml


def create_documents(resume: Resume) -> dict[TopicName, DocumentsIDs]:
    """Create documents and ids for each topic in the resume.

    Args:
        resume (Resume): Resume dictionary with topics and items.

    Raises:
        Exception: Raises an exception if an unknown topic is encountered.

    Returns:
        dict[TopicName, DocumentsIDs]: A dictionary containing documents and
            their ids for each topic.
    """

    def fix_accents(text: str) -> str:
        """
        Fixes accents in the text by replacing hex codes with their
        corresponding characters.
        """
        text = text.replace(r"\xE1", "á")
        text = text.replace(r"\xE9", "é")
        text = text.replace(r"\xED", "í")
        text = text.replace(r"\xF3", "ó")
        text = text.replace(r"\xF1", "ñ")

        return text

    documents_dict: dict[TopicName, DocumentsIDs] = dict()

    for topic_name, topic in resume.items():

        # Create an empty list of documents and ids
        documents_ids = DocumentsIDs(documents=[], metadata=dict(), ids=[])
        for item in topic:

            # Generate document for each item
            item_dump = yaml.dump(item, default_flow_style=False, width=float("inf"))
            item_parsed = fix_accents(item_dump)
            item_parsed = f"topic_name: {topic_name}\n" + item_parsed
            documents_ids["documents"].append(item_parsed)

            # Generate id for each item
            id_func = GENERATE_ID_DICT.get(topic_name, None)

            if id_func is None:
                raise Exception(f"Unknow topic: {topic_name = }, {item = }")

            id = id_func(item)

            documents_ids["ids"].append(id)

        # Finally, add the documents and ids to the main dict
        documents_dict[topic_name] = documents_ids

    return documents_dict


documents_ids = create_documents(cv)

In [ ]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry


# Define a helper to retry when per-minute quota is reached.
def is_retriable(e):
    return isinstance(e, genai.errors.APIError) and e.code in {429, 503}


class GeminiEmbeddingFunction(EmbeddingFunction):
    def __init__(self, document_mode: bool = True) -> None:
        self.embedding_task: str = (
            "retrieval_document" if document_mode else "retrieval_query"
        )
        self.model: str = "text-embedding-004"

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:

        response = client.models.embed_content(
            model=self.model,
            contents=input,
            config=types.EmbedContentConfig(task_type=self.embedding_task),
        )
        return [e.values for e in response.embeddings]

In [ ]:
import chromadb

DB_NAME = "resume"

DB_PATH = Path("..") / "data" / "03_primary" / "chromadb"
DB_PATH.parent.mkdir(parents=True, exist_ok=True)


def create_chromadb(
    embed_fn: EmbeddingFunction,
    documents_ids: DocumentsIDs,
    topic_name: str,
    language: str = LANG,
) -> chromadb.Collection:
    """Create a ChromaDB collection and add documents to it.

    This function creates a ChromaDB collection with the specified name and
    embedding function. It then adds the documents and their IDs to the
    collection.

    Args:
        embed_fn (EmbeddingFunction): The embedding function to use for the documents.
        documents_ids (dict[TopicName, DocumentsIDs]): A dictionary mapping topic names to document IDs.
        topic_name (str): The name of the topic for the collection.
        language (str): The language of the documents.

    Returns:
        chromadb.Collection: The created (or retrieved) ChromaDB collection.
    """

    chroma_client = chromadb.PersistentClient(path=str(DB_PATH))

    name_collection = (
        f"{DB_NAME}.{topic_name}"
        if language == "en"
        else f"{DB_NAME}.{language}.{topic_name}"
    )

    db = chroma_client.get_or_create_collection(
        name=name_collection, embedding_function=embed_fn
    )

    # See if the documents are already in the database
    original_db_ids = set(db.get(include=[])["ids"])
    diff_ids = set(documents_ids["ids"]) - original_db_ids

    if len(diff_ids) > 0:
        # If the documents are not already in the database, add them
        print(  # noqa: T201
            f"Adding {len(documents_ids['ids'])} documents to {name_collection}"
        )

        db.add(documents=documents_ids["documents"], ids=documents_ids["ids"])

    return db


if (docs_ids_topic := documents_ids.get(TOPIC, None)) is None:
    raise Exception(f"Topic {TOPIC} not found in documents_ids")

db = create_chromadb(
    embed_fn=GeminiEmbeddingFunction(document_mode=True),
    documents_ids=docs_ids_topic,
    topic_name=TOPIC,
)

In [ ]:
db.get(include=[])["ids"]

In [ ]:
documents_ids["work"]

In [ ]:
DESCRIPTION = """
En Clay Fintech estamos construyendo productos que integran inteligencia artificial desde el corazón. Creemos que esta tecnología puede transformar tanto nuestros procesos internos como la forma en que ayudamos a nuestros clientes.

Estamos buscando a alguien que se sume a este camino desde etapas tempranas, con muchas ganas de aprender, aportar ideas, y construir junto a nosotros. Si te interesa el mundo de la IA, los datos, y el desarrollo de software, ¡este rol es para ti!


🔧 Lo que harás

Desarrollarás herramientas internas y soluciones para clientes usando tecnologías modernas.
Colaborarás en la creación de agentes de IA, integraciones con APIs, y pipelines de datos.
Probarás nuevas formas de aplicar modelos de lenguaje (LLMs) y otros enfoques de IA.
Participarás activamente en un equipo técnico que valora la autonomía, el aprendizaje continuo y el buen feedback
 

🌱 Qué buscamos (no necesitas tener todo)

Experiencia programando en Python (proyectos personales, bootcamps, freelance o universidad).
Curiosidad real por la inteligencia artificial y el trabajo con datos.
Conocimientos básicos de bases de datos (SQL o NoSQL).
Buena comunicación, ganas de aprender y trabajar en equipo.
Bonus (no excluyente): experiencia con alguna de estas herramientas o conceptos: Langchain, LlamaIndex, embeddings o retrieval MongoDB PostgreSQL AWS (S3, Lambda, Step Functions)
🎁 Beneficios 🏖️ 5 semanas de vacaciones al año (sí, ¡cinco!)
🕓 Horario flexible y trabajo remoto desde cualquier parte de Chile
🧑‍⚕️ Seguro de salud y dental complementario
📈 Reajuste de sueldo por IPC cada 6 meses
🎂 Día libre en tu cumpleaños (y medio día libre para el de tus hijos)
📚 Tarde libre al mes para recargar energías
📝 3 días administrativos para trámites personales
👥 Ambiente de trabajo buena onda, con foco en el aprendizaje, la colaboración y el crecimiento

📍 Remoto desde Chile (también puedes trabajar desde nuestra oficina si prefieres)

Postula aquí.
"""  # noqa: W293

In [ ]:
def retrival_query(
    query: str,
    embed_fn: EmbeddingFunction,
    db: chromadb.Collection,
    max_exp: int = 5,
    window: int = 2,
) -> list[str]:
    """Query the Chroma DB and return the top passages.

    Args:
        query (str): The query string.
        embed_fn (EmbeddingFunction): The embedding function to use.
        db (chromadb.Collection): The Chroma DB collection to query.
        max_exp (int, optional): The maximum number of passages to return. Defaults to 5.
        window (int, optional): The number of passages to include in each window. Defaults to 2.

    Returns:
        list[str]: A list of passages that match the query.
    """
    n_results = db.count()
    result = db.query(query_texts=[query], n_results=min(n_results, max_exp + window))
    [all_passages] = result["documents"]
    return all_passages


all_passages = retrival_query(
    query=DESCRIPTION,
    embed_fn=GeminiEmbeddingFunction(document_mode=False),
    db=db,
    max_exp=N_MAX_EXP,
    window=2,
)

In [ ]:
supported_languages = ["es", "en"]
type Language = t.Literal[*supported_languages]

contents = ["example", "prompt", "quit_msg"]
type Content = t.Literal[*contents]


# Dictionary that stores content in different languages
content: dict[Language, dict[Content, str]] = {
    lang: dict() for lang in supported_languages
}

In [ ]:
# Quit message in different languages
content["en"]["quit_msg"] = "To exit, enter 'q'"
content["es"]["quit_msg"] = "Para salir, ingrese 'q'"

In [ ]:
# Example result for different languages

content["en"]["example"] = """
## Position Name / Title: Research Assistant
- Company Name / Business Name: CENIA
- Industry type: Information and Research
- Job Field: Education, Teaching and Research
- Sub-Area of Work: Research and Development

### Original Description

[ORIGINAL DESCRIPTION]

### Modified description

[SHORT DESCRIPTION]
- [TASK PERFORMED 1]
- ...

### Changes made

- Keywords used: [KEYWORD 1], ...
- Brief explanation of the changes made: [EXPLANATION OF CHANGES].


Shall we continue [Y/n]?
"""

content["es"]["example"] = """
## Nombre del puesto / Título: Asistente de Investigación
- Nombre de empresa / Negocio: CENIA
- Tipo de industria: Información e Investigación
- Área de trabajo: Educación, Docencia e Investigación
- Subárea de trabajo: Investigación y Desarrollo

### Descripción original

[DESCRIPCIÓN ORIGINAL]

### Descripción modificada

[PEQUEÑA DESCRIPCIÓN]
- [TAREA REALIZADA 1]
- ...

### Cambios realizados

- Palabras clave utilizadas: [PALABRA CLAVE 1], ...
- Breve explicación de los cambios realizados: [EXPLICACIÓN DE CAMBIOS]


¿Continuamos? [Y/n]
"""

In [ ]:
# Prompt for the model

content["en"]["prompt"] = r"""
You are an expert in job interviews, with deep knowledge of the applicant tracking system (ATS), and you are capable of identifying keywords from a job description. I need you to analyze the job description and the experiences listed below, and modify the experiences so that they align with the keywords from the job description. This way, the experiences will be 100% ATS-compatible.

It is important that the description is:
- ATS-friendly.
- Concise.
- Persuasive to the recruiter.
- Aligned with my personal brand.
- Written in active voice: Developed, designed, executed...

Only return what is necessary—do not add any extra or filler words.

Start by listing the identified keywords, followed by the top {n_max_exp} most relevant professional experiences, listed in reverse chronological order based on their start date. In a separate section, list the other experiences that were not selected. I ONLY WANT THE JOB TITLE AND COMPANY NAME.

EXAMPLE: "
** KEYWORDS: ** [KEYWORD 1], [KEYWORD 2], ...
** HIGHLIGHTED EXPERIENCES: **
- ([START DATE] - [END DATE]) [JOB TITLE 1], [COMPANY NAME 1]
- ...
** NON-SELECTED EXPERIENCES: **
- ([START DATE] - [END DATE]) [JOB TITLE 1], [COMPANY NAME 1]
- ...

Do you possess all the skills related to the identified keywords? Does any experience need to be modified?
"

As a reply to your message, I will let you know which experiences I have that match the keywords and whether I feel the experiences align with what I want. I will ask you to modify an experience if needed.

Once I respond, present the experience in the format shown below. IT IS IMPORTANT THAT YOU ONLY SHOW ONE EXPERIENCE AT A TIME. MENTION WHICH KEYWORDS YOU USED TO ADAPT THE EXPERIENCE AND MAKE IT MORE IMPACTFUL. There's no need to copy and paste the original experience description—you may modify or remove content if you believe it improves clarity or relevance. Be sure to mention what changes you made, what you added, and what you removed.

I will decide whether the description is appropriate and request changes if needed. Once that experience is complete, we will proceed to the next one.

EXPERIENCE FORMAT EXAMPLE: "{example}"

JOB DESCRIPTION: "{description}"

"""


content["es"]["prompt"] = r"""
Eres un experto en entrevistas de trabajo, conociendo a detalle el applicant tracking system, y eres capaz de reconocer las palabras clave a partir de la descripción de una oferta de trabajo. Necesito que, a partir de la descripción de la oferta de trabajo, y de las experiencias incluidas más abajo, seas capaz de  modificar las experiencias para que calcen con las palabras clave de la descripción. De esta manera, que las experiencias sean 100% compatibles con el applicant tracking system.

Es importante que la descripción sea:
- Compatible con ATS.
- Que tenga un contenido conciso.
- Que resulte convincente para el reclutador.
- Que sea congruente con mi marca personal.
- Que utilice la voz activa: Programé, diseñé, ejecuté...

Solo retorna lo necesario, no agregues palabras de más.

Empieza enlistando las palabras claves detectadas y enlista las {n_max_exp} experiencias más destacables, en orden cronológico decreciente con respecto a la fecha de inicio. En una sección aparte, enlista las otras experiencias que no fueron seleccionadas. INTENTA QUE EL NOMBRE SEA DECRIPTIVO, Y QUE AYUDE A DIFERENCIARSE ENTRE LAS DEMÁS EXPERIENCIAS. AGREGA UNA DESCRIPCIÓN MUY BREVE, DE UNA LÍNEA MÁXIMO.

EJEMPLO: "
** PALABRAS CLAVES: ** [PALABRA 1], [PALABRA 2], ...
** EXPERIENCIAS DESTACADAS: **
- ([FECHA DE INICIO] - [FECHA DE FIN]) [NOMBRE]: [DESCRIPCIÓN]
- ...
** EXPERIENCIAS NO SELECCIONADAS: **
- ([FECHA DE INICIO] - [FECHA DE FIN]) [NOMBRE]: [DESCRIPCIÓN]
- ...

¿Posees todas las habilidades de las palabras claves? ¿Es necesario modificar
alguna experiencia?
"

Como respuesta a tu mensaje, te responderé cuáles son las experiencias que tengo con las palabras claves, y también si considero que las experiencias laborales están acorde a lo que deseo. Te pediré cambiar alguna experiencia si se da el caso. NO RESPONDAS LAS ÚLTIMAS PREGUNTAS DEL EJEMPLO, SOLO DEBES MOSTRAR LAS PREGUNTAS. YO RESPONDERÉ A LAS PREGUNTAS.

Una vez que haya respondido, presentame la experiencia en el formato que te entrego más abajo. ES IMPORTANTE QUE MUESTRES SOLO UNA ÚNICA EXPERIENCIA. MENCIONA QUÉ PALABRAS CLAVE UTILIZASTE PARA MODIFICAR LA OFERTA Y DARLE MAYOR IMPACTO. No es necesario copiar y pegar la descripción de la experiencia original, puedes modificar o quitar experiencias si lo consideras necesario. Recuerda mencionar qué cambios hiciste, qué agregaste y qué quitaste.

Yo decidiré si la descripción es adecuada, donde te pediré modificaciones si es necesario. Una vez que termine con esa experiencia, procederemos a ver la siguiente experiencia.

EJEMPLO DE FORMATO DE EXPERIENCIA: "{example}"

DESCRIPCIÓN DE LA OFERTA: "{description}"

"""

In [ ]:
content_lang = content[LANG]

prompt = content_lang["prompt"].format(
    example=content_lang["example"], description=DESCRIPTION, n_max_exp=N_MAX_EXP
)

for passage in all_passages:
    passage_oneline = passage.replace("\n", "  \n")
    prompt += "EXPERIENCE: " + passage_oneline + "\n"

In [ ]:
config = types.GenerateContentConfig(temperature=0.8, top_p=0.95, top_k=30)
chat = client.chats.create(model="gemini-2.0-flash", history=[], config=config)

response = chat.send_message(prompt)

Markdown(response.text)

In [ ]:
while True:
    print(content_lang["quit_msg"])  # noqa: T201
    msg: str = input("> ")
    if msg.lower() == "q":
        break
    if not msg:
        msg = "Y"
    print("")  # noqa: T201
    response = chat.send_message(msg)
    print(response.text)  # noqa: T201